In [40]:
!nvidia-smi

Tue May 10 15:02:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    74W / 149W |    986MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Data Loading

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df=pd.DataFrame(
    columns = ['build_Failed', 'gh_is_pr', 'git_prev_commit_resolution_status',
       'gh_team_size', 'gh_num_commit_comments', 'git_diff_src_churn',
       'git_diff_test_churn', 'gh_diff_files_added', 'gh_diff_files_deleted',
       'gh_diff_files_modified', 'gh_diff_tests_added',
       'gh_diff_tests_deleted', 'gh_diff_src_files', 'gh_diff_doc_files',
       'gh_diff_other_files', 'gh_sloc', 'gh_test_lines_per_kloc',
       'gh_test_cases_per_kloc', 'gh_asserts_cases_per_kloc', 'tr_build_id',
       'gh_build_started_at'],
    dtype='object')

path = '/mnt/d/PFE/Code/CI-Failing-Builds-Detection/dataset'
path = '/content/drive/MyDrive/CI/replicating-ets/DL-CIBuild/dataset'

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename[-4:]==".csv":
            df = pd.concat([df, pd.read_csv(os.path.join(dirname, filename))])

X = df.iloc[:,1:19]
y = df.iloc[:,0].astype(int)

from sklearn.model_selection import train_test_split
X_train, X_val , y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=True, stratify=y, random_state=42) # keep ratio of classes in split

### Modifiable Decision Tree model 

In [43]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import f1_score 

class modDecisionTree:
    """
    Represents the classification model
    based on sklearn implementation with added methods for modifying single nodes
    nodes are indexed depth first
    """


    def __init__(self, max_depth=3, random_state=42):
        # need to add init of other hyper-param
        self.max_depth = max_depth
        self.model = DecisionTreeClassifier(max_depth=max_depth, splitter='random', random_state=random_state)
    

    def fit(self, X_train, y_train, columns_names):
        self.model.fit(X_train, y_train)
        self.tree = self.model.tree_
        self.n_nodes = self.tree.node_count       # nbr of nodes
        self.features = self.tree.feature         # list of each node's feature
        self.thresholds = self.tree.threshold     # list of each node's threshold
        self.columns_names = columns_names
        self.features_names = [list(self.columns_names)[i] for i in self.features]
        self.nodes_type = self.get_nodes_type()  # a node is either a split node or a terminal node/leaf


    def evaluate(self, X_val, y_val):
        y_pred = self.model.predict(X_val)
        metrics = {}
        metrics['F1'] = f1_score(y_pred ,y_val)
        return metrics


    def feature_importance(self):
        feat_imp = []
        for name, importance in zip(self.features_names , self.model.feature_importances_):
            feat_imp.append((name, importance))
        feat_imp.sort(key=lambda t:t[1], reverse=True)
        return feat_imp


    def plot_tree(self):
        plt.figure(figsize=(15,10))  # set plot size (denoted in inches)
        tree.plot_tree(self.model, fontsize=10, class_names=['pass','fail'])
        plt.show()


    def get_nodes_type(self):
        children_left = self.tree.children_left
        children_right = self.tree.children_right
        node_depth = np.zeros(shape=self.n_nodes, dtype=np.int64)
        is_leaves = np.zeros(shape=self.n_nodes, dtype=bool)
        stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
        while len(stack) > 0:
            # 'pop' ensures each node is only visited once
            node_id, depth = stack.pop()
            node_depth[node_id] = depth
            # If the left and right child of a node is not the same we have a split node
            is_split_node = children_left[node_id] != children_right[node_id]
            # If a split node, append left and right children and depth to `stack`
            # so we can loop through them
            if is_split_node:
                stack.append((children_left[node_id], depth + 1))
                stack.append((children_right[node_id], depth + 1))
            else:
                is_leaves[node_id] = True
        return is_leaves


    def node_is_leaf(self, node):
        """Returns whether a node is a leaf (terminal node) or a split node"""
        return self.nodes_type[node]==1


    def set_node_threshold(self, node, value):
        if node>=self.tree.node_count:
            print("Error: selected node id is not in the tree.")
            return
        if self.node_is_leaf(node):
            print("Error: can't change a leaf node's threshold.")
            return
        self.thresholds[node] = value 


    def set_node_feature(self, node, feat_index=None, feat_name=None):
        if node>=self.tree.node_count:
            print("Error: selected node id is not in the tree.")
            return
        if self.node_is_leaf(node):
            print("Error: can't change a terminal node's feature.")
            return
        # convert feature name to index if supplied with name
        if feat_index==None:
            feat_index = self.features_names.index(feat_name)
        self.features[node] = feat_index

### State Generation

In [44]:
def tree_convolution(model, features, thresholds, node=0):
    """
        convolution of each 3 nodes, with overlapping => child of 1 subtree is parent of the next
        => after each conv, slit nodes' featuress and thresholdss are updated and terminal nodes are removed
        returns new features,thresholds values as lists
    """
    visited = []
    queue = []
    visited.append(node)
    queue.append(node)
    new_features = features
    new_thresholds = thresholds
    #replacing leaf default values -2 by 0 for conv
    for node in range(model.n_nodes): 
        if new_features[node]==-2:
            new_features[node]=0
        if new_thresholds[node]==-2:
            new_thresholds[node]=0
    # BFS traversal
    while queue:
        node = queue.pop(0) 
        left_node = model.tree.children_left[node]
        right_node = model.tree.children_right[node]
        #print (node, left_node, right_node, end = "\n")
        if left_node not in visited:
            visited.append(left_node)
            queue.append(left_node)
        if right_node not in visited:
            visited.append(right_node)
            queue.append(right_node)

        # define kernel here
        if features[left_node]==0 and features[right_node]==0: # both child nodes are leaves
            new_node_features = 0
            new_node_thresholds = 0
        elif features[left_node]==0: # left node is leaf
            new_node_features = (features[node]+features[right_node])/2
            new_node_thresholds = (thresholds[node]+thresholds[right_node])/2
        elif features[right_node]==0: # right node is leaf
            new_node_features = (features[node]+features[left_node])/2
            new_node_thresholds = (thresholds[node]+thresholds[left_node])/2
        else:
            new_node_features = (features[node]+features[left_node]+features[right_node])/3
            new_node_thresholds = (thresholds[node]+thresholds[left_node]+thresholds[right_node])/3
        # save new values
        new_features[node] = new_node_features # will auto convert to int 
        new_thresholds[node] = new_node_thresholds

    return new_features, new_thresholds


def flatten(features, thresholds):
    return torch.cat((torch.FloatTensor(features), torch.FloatTensor(thresholds))).to(device)


def generate_state(model, features, thresholds, nbr_of_conv):
    features, thresholds = features.copy(), thresholds.copy()
    for _ in range(nbr_of_conv):
        new_features, new_thresholds = tree_convolution(model, features, thresholds) # model must be fitted => add condition?
        features, thresholds = new_features, new_thresholds
    return flatten(features, thresholds)


### Reinforcement Learning

In [45]:
import numpy as np
import random

from collections import namedtuple, deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [46]:
class ThresholdsNetwork(nn.Module):
    """Network that will predict the new thresholds vector given a state."""


    def __init__(self, state_size, threshold_vector_size, seed, hidden_size=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each action
            seed (int): Random seed
            hidden_size (int): Number of nodes in hidden layers
        """
        super(ThresholdsNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)     
        self.fc1 = nn.Linear(state_size, hidden_size)
        #self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, threshold_vector_size)


    def forward(self, state):
        x = F.relu(self.fc1(state), inplace=True)
        #x = F.relu(self.fc2(x), inplace=True)
        out = self.fc3(x)
        return out
    

    def get_thresholds_vector(self, state):
        threshold_vector = self.forward(state).to(device)
        return threshold_vector

In [47]:
class AttributeNetwork(nn.Module):
    """Network that will select a new attribute for a tree node given the environment state and thresholds vector"""

    def __init__(self, state_size, threshold_vector_size, number_of_attributes, seed, hidden_size=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each threshold vector
            seed (int): Random seed
            hidden_size (int): Number of nodes in the network layers

        """
        super(AttributeNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size+threshold_vector_size, hidden_size)
        #self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, number_of_attributes)


    def forward(self, state, threshold_vector):
        """Build a critic (value) network that maps (state, threshold_vector) pairs -> Q-values."""
        x = torch.cat((state, threshold_vector), dim=-1)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        return self.fc3(x)


    def get_attributes_vector(self, state, threshold_vector, Xe_vect=False):
        if Xe_vect:
            # the input threshold vector is Xe instead of X; for the calc of target yb
            attributes_vector = self.forward(state, threshold_vector)

        else:
            # decompose (st,X) input into (st,Xe_k) for each k 
            attributes_vector = torch.zeros(len(threshold_vector)).to(device)
            X = threshold_vector
            for k in range(len(X)):
                Xe = torch.zeros(len(X)).to(device)    
                Xe[k] = X[k]
                q_vect = self.forward(state,Xe)
                attributes_vector[k] = q_vect[k]
        return attributes_vector

In [48]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""


    def __init__(self, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)


    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)


    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        states = torch.vstack([e.state for e in experiences if e is not None]).float().to(device)
        actions = torch.vstack([torch.tensor(e.action) for e in experiences if e is not None]).float().to(device)
        rewards = torch.vstack([torch.tensor(e.reward) for e in experiences if e is not None]).float().to(device)
        next_states = torch.vstack([e.next_state for e in experiences if e is not None]).float().to(device)
        dones = torch.vstack([torch.tensor(e.done) for e in experiences if e is not None]).float().to(device)
        return zip(states, actions, rewards, next_states, dones)


    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
    

In [49]:
class Agent():
    """Interacts with and learns from the environment."""
    

    def __init__(self, state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            threshold_vector_size (int): dimension of each threshold vector
            random_seed (int): random seed
            add rest
        """
        self.state_size = state_size
        self.threshold_vector_size = threshold_vector_size
        self.number_of_attributes = number_of_attributes
        self.seed = random.seed(random_seed)
        print("Using: ", device)

        # actor Network 
        self.ThresholdsNetwork = ThresholdsNetwork(state_size, threshold_vector_size, random_seed, hidden_size).to(device)
        self.optimizer_ThresholdsNetwork = optim.Adam(self.ThresholdsNetwork.parameters(), lr=LR_ACTOR)     
        
        # critic Network  
        self.AttributeNetwork = AttributeNetwork(state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size).to(device)
        self.optimizer_AttributeNetwork = optim.Adam(self.AttributeNetwork.parameters(), lr=LR_CRITIC, weight_decay=0)

        # Replay memory
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, random_seed)
        

    def act(self, state, eps=0.2):
        """Returns actions for given state as per current policy."""
        # greedy epsilon with param eps
        thresholds_vector = self.ThresholdsNetwork.get_thresholds_vector(state)

        p = np.random.random() 
        if p<eps:
            index_selected_attribute = random.choice(range(self.number_of_attributes))
        else:
            attributes_vector = self.AttributeNetwork.get_attributes_vector(state, thresholds_vector)
            index_selected_attribute = torch.argmax(attributes_vector)
        action = (index_selected_attribute, thresholds_vector.squeeze(0)[index_selected_attribute])

        return action


    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # if enough samples are available in memory
        # sample a minibatch and learn/update networks
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)


    def learn(self, experiences, gamma):
        """Updates the two neural networks using given batch of experience tuples.
        Critic_loss = 
        Actor_loss = 
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        Q_loss, X_loss = [], []
        for experience in experiences: # for b in B
            state, action, reward, next_state, done = experience
            sb = state
            k_act = int(action[0].item())
            rb = reward
            sb1 = next_state # S_{b+1}
            Xb1 = self.ThresholdsNetwork.get_thresholds_vector(sb1) # X_{b+1}
            Xb = self.ThresholdsNetwork.get_thresholds_vector(sb) # X_{b}
            # yb calc
            # get max_k(Qq)
            Qq = []
            for k in range(self.number_of_attributes):
                # get Xe_{b+1}
                Xeb1k = torch.zeros(len(Xb1)).to(device)
                Xeb1k[k] = Xb1[k]
                qek = torch.max(self.AttributeNetwork.get_attributes_vector(sb1, Xeb1k, Xe_vect=True))
                Qq.append(qek)
            maxQq = torch.max(torch.Tensor(Qq))

            if done: # terminal node
                yb = rb
            else:
                yb = rb + gamma * maxQq

            # compute losses for single transitions

            # Q loss
            xebk = torch.zeros(len(Xb)).to(device)
            xebk[k_act] = Xb[k_act]
            Q_loss.append(yb-self.AttributeNetwork.get_attributes_vector(sb, xebk, Xe_vect=True)[k_act])
            
            # X loss
            sum_Qq = 0
            for k in range(self.number_of_attributes):
                Xebk = torch.zeros(len(Xb)).to(device)
                Xebk[k] = Xb[k] 
                # Xe_{b,k}
                qek = self.AttributeNetwork.get_attributes_vector(sb1, Xebk, Xe_vect=True)
                sum_Qq += qek[k]
            X_loss.append(-sum_Qq)

        # compute losses as expectation over the experiences batch and update networks

        # update thresholds network
        # Compute loss
        loss_thresholds_network = torch.mean(torch.Tensor(Q_loss))
        loss_thresholds_network.requires_grad_()
        # Minimize the loss
        self.optimizer_ThresholdsNetwork.zero_grad()
        loss_thresholds_network.backward()
        self.optimizer_ThresholdsNetwork.step()

        # update attribute network
        # Compute loss
        loss_attribute_network = torch.mean(torch.Tensor(X_loss))
        loss_attribute_network.requires_grad_()
        # Minimize the loss
        self.optimizer_AttributeNetwork.zero_grad()
        loss_attribute_network.backward()
        self.optimizer_AttributeNetwork.step()


In [50]:
prev_metric = 0

def env_step(model, node, action):
    # update tree
    model.set_node_feature(node, feat_index=action[0])
    model.set_node_threshold(node, value=action[1])
    next_state = generate_state(model, model.features, model.thresholds, nbr_of_conv)
    # calc reward
    global prev_metric
    metrics = model.evaluate(X_val, y_val)
    current_metric = metrics['F1']
    reward = current_metric-prev_metric 
    prev_metric = current_metric
    done = 0
    if model.node_is_leaf(node):
        done = 1
        
    info = 0

    return next_state, reward, done, info 

In [ ]:
from tqdm import tqdm

t0 = time.time()

# hyper-parameters
HIDDEN_SIZE = 128
BUFFER_SIZE = int(1e6)
BATCH_SIZE = 64
LR_ACTOR = 5e-4 # learning rate
LR_CRITIC = 5e-4
GAMMA = 0.99    # reward calc
nbr_of_conv = 0
max_depth = 15
n_episodes = 200

scores_deque = deque(maxlen=100)
writer = SummaryWriter("runs/")
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

model = modDecisionTree(max_depth=max_depth)
model.fit(X_train, y_train, df.columns)

state = generate_state(model, model.features, model.thresholds, nbr_of_conv)
state_size = len(state)
number_of_attributes = X_train.shape[1]
threshold_vector_size = X_train.shape[1]
print(f'tree depth={max_depth}, state size={state_size}, number of attribute={number_of_attributes}')

agent = Agent(state_size, threshold_vector_size, number_of_attributes, seed, HIDDEN_SIZE)

for i_episode in range(1, n_episodes+1):
    # state reset => new DT model
    model = modDecisionTree(max_depth=max_depth)
    model.fit(X_train, y_train, df.columns)
    state = generate_state(model, model.features, model.thresholds, nbr_of_conv)
    avg_score = 0
    for t in tqdm(range(model.n_nodes)):
        if model.node_is_leaf(t):
            continue
        action = agent.act(state)
       # print(f"node={t}: {model.features[t]}<{model.thresholds[t]} => {action[0]}<{action[1]}")
        next_state, reward, done, info = env_step(model, t, action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        avg_score += reward
        if done:
            break
    avg_score /= model.n_nodes

    scores_deque.append(avg_score)
    writer.add_scalar("Reward", avg_score, i_episode) # for TensorBoard
    print('\rEpisode {} Score: {:.2f}'.format(i_episode, avg_score, end=""))

print(f"final score: {reward}")
t1 = time.time()
print("training took {} min!".format((t1-t0)/60))


tree depth=15, state size=12046, number of attribute=18
Using:  cuda:0


  2%|▏         | 147/6023 [00:20<1:07:03,  1.46it/s]

In [ ]:
print(model.feature_importance())
model.plot_tree()
model.features_names[15]